# Практическое занятие №4

П.Н. Советов, РТУ МИРЭА

**Задача 2**

Использование встроенных функций.

1. Напишите код, который выведет на экране все переменные объекта произвольного пользовательского класса.
1. Напишите код, который по имени метода, заданному строкой, вызовет этот метод в объекте некоторого пользовательского класса.

In [1]:
class Class:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def method_1(self):
        return self.x * self.y
        
    def method_2(self):
        return self.x + self.y


obj = Class(5, 10)
print(obj.x, obj.y)

s = 'method_1'
getattr(obj, s)()

5 10


50

**Задача 5**

Язык HTML-тегов с помощью менеджера контекста.

Реализовать классы для выполнения следующего примера:

```Python
html = HTML()
with html.body():
    with html.div():
        with html.div():
            html.p('Первая строка.')
            html.p('Вторая строка.')
        with html.div():
            html.p('Третья строка.')
print(html.get_code())
```

Результат:

```HTML
<body>
<div>
<div>
<p>Первая строка.</p>
<p>Вторая строка.</p>
</div>
<div>
<p>Третья строка.</p>
</div>
</div>
</body>
```

In [2]:
class HTML:
    s = ""

    def get_code(self):
        return (self.s)

    class body:
        def __enter__(self):
            HTML.s += "<body>\n"

        def __exit__(self, exc_type, exc_val, exc_tb):
            HTML.s += "</body>\n"

    class div:
        def __enter__(self):
            HTML.s += "<div>\n"

        def __exit__(self, exc_type, exc_val, exc_tb):
            HTML.s += "</div>\n"

    class p(object):
        def __init__(self, ps):
            HTML.s += "<p>" + ps + "</p>\n"


html = HTML()
with html.body():
    with html.div():
        with html.div():
            html.p('Первая строка.')
            html.p('Вторая строка.')
        with html.div():
            html.p('Третья строка.')
print(html.get_code())

<body>
<div>
<div>
<p>Первая строка.</p>
<p>Вторая строка.</p>
</div>
<div>
<p>Третья строка.</p>
</div>
</div>
</body>



**Задача 8**

Визуализатор AGI-графики из старых компьютерных игр комании Sierra.

В старых играх от Sierra (например, в [King's Quest](https://www.mobygames.com/game/kings-quest) 1984 года) фоновая графика была представлена в виде последовательности [команд](https://wiki.scummvm.org/index.php?title=AGI/Specifications/Pic). В целом, результат очень напоминал векторную графику. В оригинале использовалось разрешение 160x200 пикселей, но можно попробовать перерисовать картинки из King's Quest в высоком разрешении.

1. Реализуйте разбор команд из граф. файлов в каталоге `data/pic.*`.
2. Нарисуйте средствами `tkinter` результат в высоком разрешении без заливки экрана. Учитывайте, что игре используется 2 типа экранов: обычный и экран приоритетов.
3. (повышенной сложности) Придумайте способ добавить корректную цветовую заливку областей экрана.

Заготовка:

In [3]:
from pathlib import Path
import tkinter as tk
SCALE_X = 6
SCALE_Y = 4

COLORS = [
    (0, 0, 0),
    (0, 0, 168),
    (0, 168, 0),
    (0, 168, 168),
    (168, 0, 0),
    (168, 0, 168),
    (168, 84, 0),
    (168, 168, 168),
    (84, 84, 84),
    (84, 84, 252),
    (84, 252, 84),
    (84, 252, 252),
    (252, 84, 84),
    (252, 84, 252),
    (252, 252, 84),
    (252, 252, 252)
]


def change_color():
    global color
    new_color = yield
    color = new_color
    nb = yield
    return nb


def stop_drawing():
    while True:
        nb = yield
        if nb == 240:
            return 240


def draw_conner_y():
    x1 = yield
    y1 = yield
    while True:
        y2 = yield
        if y2 >= 240:
            return y2
        draw_line([(x1, y1), (x1, y2)], color)
        y1 = y2
        x2 = yield
        if x2 >= 240:
            return x2
        draw_line([(x1, y1), (x2, y1)], color)
        x1 = x2


def draw_conner_x():
    global color
    x1 = yield
    y1 = yield
    while True:
        x2 = yield
        if x2 >= 240:
            return x2
        draw_line([(x1, y1), (x2, y1)], color)
        x1 = x2
        y2 = yield
        if y2 >= 240:
            return y2
        draw_line([(x1, y1), (x1, y2)], color)
        y1 = y2


def draw_long_line():
    global color
    x1 = yield
    y1 = yield
    while True:
        x2 = yield
        if x2 >= 240:
            return x2
        y2 = yield
        if y2 >= 240:
            return y2
        draw_line([(x1, y1), (x2, y2)], color)
        x1 = x2
        y1 = y2


def draw_rel_line():
    global color
    x1 = yield
    y1 = yield
    while True:
        disp = yield

        if disp >= 240:
            return disp
        dx = ((disp & 240) >> 4) & 15
        dy = disp & 15
        if dx & 8:
            dx = (-1) * (dx & 7)
        if dy & 8:
            dy = (-1) * (dy & 7)
        draw_line([(x1, y1), (x1 + dx, y1 + dy)], color)
        x1 += dx
        y1 += dy


def fill():
    nb = yield
    while nb // 10 != 24:
        nb = yield


def main_gen():
    nb = yield
    while True:
        if nb == 255:
            return
        elif nb == 240:
            nb = yield from change_color()
        elif nb == 241:
            nb = yield from stop_drawing()
        elif nb == 242:
            nb = yield
        elif nb == 244:
            nb = yield from draw_conner_y()
        elif nb == 245:
            nb = yield from draw_conner_x()
        elif nb == 246:
            nb = yield from draw_long_line()
        elif nb == 247:
            nb = yield from draw_rel_line()
        else:
            nb = yield


def draw(pic):
    g = main_gen()
    next(g)
    for nb in pic:
        g.send(nb)


def draw_line(coords, color_index):
    canvas.create_line(*[(x * SCALE_X, y * SCALE_Y) for x, y in coords],
                       fill='#%02x%02x%02x' % COLORS[color_index], width=4)


color = 0
pic = Path('data/PIC.44').read_bytes()
print([hex(i) for i in pic])
canvas = tk.Canvas(width=160 * SCALE_X, height=170 * SCALE_Y)
canvas.pack()
draw(pic)
tk.mainloop()

FileNotFoundError: [Errno 2] No such file or directory: 'data\\PIC.44'